# TruBot

Example setup and monitoring of a conversational bot with context made up of the TruEra website.

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.resolve()))

# Uncomment for more debugging printouts.
"""
import logging
root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

"\nimport logging\nroot = logging.getLogger()\nroot.setLevel(logging.DEBUG)\n\nhandler = logging.StreamHandler(sys.stdout)\nhandler.setLevel(logging.DEBUG)\nformatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')\nhandler.setFormatter(formatter)\nroot.addHandler(handler)\n"

In [2]:
from pprint import PrettyPrinter
from typing import Sequence

from IPython.display import JSON
# imports from langchain to build app
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import LLMChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts.chat import ChatPromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.prompts.chat import PromptTemplate
from langchain.vectorstores import DocArrayHnswSearch
import numpy as np

# Imports main tools:
from trulens_eval import Feedback
from trulens_eval import Huggingface
from trulens_eval import Query
from trulens_eval import Tru
from trulens_eval import tru_feedback
from trulens_eval import TruChain
from trulens_eval.keys import *
from trulens_eval.schema import Query
from trulens_eval.tru_db import Record
from trulens_eval.tru_feedback import Feedback
from trulens_eval.tru_feedback import Huggingface
from trulens_eval.util import Step
from trulens_eval.utils.langchain import WithFeedbackFilterDocuments

# if using Pinecone vectordb:
# from langchain.vectorstores import Pinecone
# import pinecone

tru = Tru()
pp = PrettyPrinter()

tru.reset_database()

KEY SET: OPENAI_API_KEY
KEY SET: PINECONE_API_KEY
KEY SET: PINECONE_ENV
KEY SET: HUGGINGFACE_API_KEY
KEY SET: SLACK_TOKEN
KEY SET: SLACK_SIGNING_SECRET
KEY SET: COHERE_API_KEY


/opt/homebrew/Caskroom/miniconda/base/envs/py38_trulens/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name = "gpt-3.5-turbo"
chain_id = "TruBot"

# Pinecone configuration if using pinecone.
# pinecone.init(
#    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
#    environment=PINECONE_ENV  # next to api key in console
#)
#docsearch = Pinecone.from_existing_index(
#    index_name="llmdemo", embedding=embedding
#)

# LLM for completing prompts, and other tasks.
llm = OpenAI(temperature=0, max_tokens=256)

# Construct feedback functions.

hugs = tru_feedback.Huggingface()
openai = tru_feedback.OpenAI()

# Language match between question/answer.
f_lang_match = Feedback(hugs.language_match).on(
    text1=Query.RecordInput, text2=Query.RecordOutput
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on(
    prompt=Query.RecordInput, response=Query.RecordOutput
)

# Question/statement relevance between question and each context chunk.
f_qs_relevance = tru_feedback.Feedback(openai.qs_relevance).on(
    question=Query.RecordInput,
    statement=Query.Record.model.combine_docs_chain._call.args.inputs.
    input_documents[:].page_content
).aggregate(np.min)

def new_conversation(
    lang_prompt_fix: bool = False,
    context_prompt_fix: bool = False,
    context_filter_fix: bool = False,
    feedbacks: Sequence[Feedback] = None
):
    """
    Create a chain for a new conversation (blank memory). Set flags to enable
    adjustments to prompts or add context filtering.
    """
    
    assert not(lang_prompt_fix and context_prompt_fix), "Cannot use both prompt fixes at the same time."

    # Embedding needed for Pinecone vector db.
    embedding = OpenAIEmbeddings(model='text-embedding-ada-002')  # 1536 dims

    # Conversation memory.
    memory = ConversationSummaryBufferMemory(
        max_token_limit=650,
        llm=llm,
        memory_key="chat_history",
        output_key='answer'
    )

    # Pinecone alternative. Requires precomputed 'hnswlib_truera' folder.
    docsearch = DocArrayHnswSearch.from_params(
        embedding=embedding,
        work_dir='hnswlib_trubot',
        n_dim=1536,
        max_elements=1024
    )
    retriever = docsearch.as_retriever()

    # Better contexts fix, filter contexts with relevance:
    if context_filter_fix: 
        retriever = WithFeedbackFilterDocuments.of_retriever(
            retriever=retriever, feedback=f_qs_relevance, threshold = 0.5
        )

    # Conversational chain puts it all together.
    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        return_source_documents=True,
        memory=memory,
        get_chat_history=lambda a: a,
        max_tokens_limit=4096
    )

    # Need to copy these otherwise various chains will feature templates that
    # point to the same objects.
    chain.combine_docs_chain.llm_chain.prompt = \
        chain.combine_docs_chain.llm_chain.prompt.copy()
    chain.combine_docs_chain.document_prompt = \
        chain.combine_docs_chain.document_prompt.copy()

    # Language mismatch fix:
    if lang_prompt_fix:
        chain.combine_docs_chain.llm_chain.prompt.template = \
            "Use the following pieces of context to answer the question at the end " \
            "in the same language as the question. If you don't know the answer, " \
            "just say that you don't know, don't try to make up an answer.\n\n" \
            "{context}\n\n" \
            "Question: {question}\n" \
            "Helpful Answer: "

    # Poor contexts fix using prompts:
    elif context_prompt_fix:
        chain.combine_docs_chain.llm_chain.prompt.template = \
            "Use only the relevant contexts to answer the question at the end " \
            ". Some pieces of context may not be relevant. If you don't know the answer, " \
            "just say that you don't know, don't try to make up an answer.\n\n" \
            "Contexts: \n{context}\n\n" \
            "Question: {question}\n" \
            "Helpful Answer: "
        chain.combine_docs_chain.document_prompt.template = "\tContext: {page_content}"

    # Trulens instrumentation.
    tc = Tru().Chain(chain=chain, feedbacks=feedbacks, verbose=True)

    return tc

huggingface api: 0requests [00:00, ?requests/s]

In [4]:
# Instantiate a chain

tc = new_conversation(
    #feedbacks=[f_lang_match], # any feedbacks specified here will be executed automatically
    context_filter_fix=True
)

✅ chain chain_hash_dc4dc7ebd5cb44df31e1757f0b7f6c17 -> default.sqlite


In [5]:
# tc.dict()

In [6]:
# Call the chain

res, record = tc.call_with_record("Who is Shayak?")

✅ record record_hash_06081d1caa00b748a55f247c2e69db7a from chain_hash_dc4dc7ebd5cb44df31e1757f0b7f6c17 -> default.sqlite


In [7]:
record.dict()

{'record_id': 'record_hash_06081d1caa00b748a55f247c2e69db7a',
 'chain_id': 'chain_hash_dc4dc7ebd5cb44df31e1757f0b7f6c17',
 'cost': {'n_tokens': 389, 'cost': 0.0077800000000000005},
 'perf': {'start_time': datetime.datetime(2023, 6, 10, 20, 0, 15, 328914),
  'end_time': datetime.datetime(2023, 6, 10, 20, 0, 23, 847941)},
 'ts': datetime.datetime(2023, 6, 10, 20, 0, 23, 848057),
 'tags': '',
 'main_input': 'Who is Shayak?',
 'main_output': ' Shayak is a computer scientist who obtained his PhD in Computer Science from Carnegie Mellon University and BTech in Computer Science from the Indian Institute of Technology, Delhi. He has been building production grade machine learning models for algorithmic trading for 10 years and has been leading research to make machine learning and big data systems more explainable, privacy compliant, and fair. His research at Carnegie Mellon University introduced a number of pioneering breakthroughs to the field of explainable AI.',
 'main_error': 'None',
 'ca

In [8]:
# Evaluate a feedback function manually.

fres = f_qs_relevance.run(chain=tc, record=record)

In [9]:
from trulens_eval.util import json_str_of_obj
from trulens_eval.util import jsonify

pp.pprint(list(c.dict() for c in fres.calls))

[{'args': {'question': 'Who is Shayak?',
           'statement': 'When Shayak started building production grade machine '
                        'learning models for algorithmic trading 10 years ago, '
                        'he realized the need for putting the ‘science’ back '
                        'in ‘data science’. Since then, he has been building '
                        'systems and leading research to make machine learning '
                        'and big data systems more explainable, privacy '
                        'compliant, and fair. Shayak’s research at Carnegie '
                        'Mellon University introduced a number of pioneering '
                        'breakthroughs to the field of explainable AI. Shayak '
                        'obtained his PhD in Computer Science from Carnegie '
                        'Mellon University and BTech in Computer Science from '
                        'the Indian Institute of Technology, Delhi.'},
  'ret': 1.0},
 {'

In [10]:
ress = tru.run_feedback_functions(record=record, feedback_functions=[f_qs_relevance])

In [11]:
for res in ress:
    display(res.dict())

{'feedback_result_id': 'feedback_result_hash_cce3c029211f0f67025fdd20f102ad2c',
 'record_id': 'record_hash_06081d1caa00b748a55f247c2e69db7a',
 'chain_id': 'chain_hash_dc4dc7ebd5cb44df31e1757f0b7f6c17',
 'feedback_definition_id': 'feedback_definition_hash_d7f11346030683ccf302a042362b5c0f',
 'last_ts': datetime.datetime(2023, 6, 10, 20, 1, 2, 565799),
 'status': <FeedbackResultStatus.DONE: 'done'>,
 'cost': {'n_tokens': 0, 'cost': 0.0},
 'tags': '',
 'name': 'qs_relevance',
 'calls': [{'args': {'question': 'Who is Shayak?',
    'statement': 'When Shayak started building production grade machine learning models for algorithmic trading 10 years ago, he realized the need for putting the ‘science’ back in ‘data science’. Since then, he has been building systems and leading research to make machine learning and big data systems more explainable, privacy compliant, and fair. Shayak’s research at Carnegie Mellon University introduced a number of pioneering breakthroughs to the field of explaina